In [1]:
import pandas as pd

In [2]:
bu = pd.read_csv("yelp_business.csv")

In [3]:
# Removes all values that are not in Santa Barbara
bu = bu[bu["city"] == "Santa Barbara"]

In [4]:
# Removes all values that are not restaurants
bu["categories"] = bu["categories"].astype("str")
bu["categories"] = bu["categories"].apply(lambda x: set(x.split(", ")))

word_list = ["Restaurants", "Food", "Restaurant"]
bu["categories"] = bu["categories"].apply(lambda x: x.intersection(word_list))
bu = bu[bu["categories"] != set()]

In [5]:
# Removes all values that are not open
bu = bu[bu["is_open"] == 1]

In [6]:
# Drops unnessescary columns and does some formatting 
bu = bu.drop(columns=['_id', 'city', 'is_open', 'state', 'categories'])
bu["attributes.RestaurantsTakeOut"].fillna("False", inplace = True)
bu["attributes.RestaurantsTakeOut"] = bu["attributes.RestaurantsTakeOut"].replace("None", "False")

In [7]:
bu = bu.rename(columns={"attributes.RestaurantsTakeOut":"take_out"})

In [8]:
bu.to_csv('yelp_business_clean.csv', index=False)

In [9]:
bu.head()

,address,take_out,business_id,latitude,longitude,name,review_count,stars
85,"131 Anacapa St, Ste C",True,IDtLPgUrqorrpqSLdfMhZQ,34.414445,-119.690672,Helena Avenue Bakery,389,4.0
141,230 Stearns Wharf,True,SZU9c8V2GuREDN5KgyHFJw,34.408715,-119.685019,Santa Barbara Shellfish Company,2404,4.0
431,2733 De La Vina St,True,ifjluUv4VASwmFqEp8cWlQ,34.436236,-119.726147,Marty's Pizza,64,4.0
691,528 Anacapa St,True,-kY_HDP7IMvGl-kBIZVU4A,34.418994,-119.695068,Dune Coffee Roasters - Anacapa,320,4.0
946,116 Santa Barbara St,True,-ujBP1Dw0j1-Ffaz97-LXQ,34.415747,-119.688480,Lama Dog Tap Room,238,4.0


In [10]:
re = pd.read_csv("yelp_review.csv")

In [11]:
re.head()

,_id,business_id,review_id,stars
0,6410f4de59e3fd5878fd30aa,XQfwVwDr-v0ZS3_CbbE5Xw,KU_O5udG6zpxOg-VcAEodg,3
1,6410f4de59e3fd5878fd30ab,7ATYjTIgM3jUlt4UM3IypQ,BiTunyQ73aT9WBnpR9DZGw,5
2,6410f4de59e3fd5878fd30ac,YjUWPpI6HXG530lwP-fb2A,saUsX_uimxRlCVr67Z4Jig,3
3,6410f4de59e3fd5878fd30ad,kxX2SOes4o-D3ZQBkiMRfA,AqPFMleE6RsU23_auESxiA,5
4,6410f4de59e3fd5878fd30ae,e4Vwtrqf-wpJfwesgvdgxQ,Sx8TMOWLNuJBWer-0pcmoA,4


In [12]:
re = re.drop(columns=['_id'])

In [13]:
re = pd.merge(bu, re, how ='inner', on =['business_id']).drop(columns=["address", "take_out", "latitude", "longitude", "name", 
                                                               "review_count", "stars_x"]).rename(columns={"stars_y":"stars"})

In [17]:
re.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147588 entries, 0 to 147587
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  147588 non-null  object
 1   review_id    147588 non-null  object
 2   stars        147588 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.5+ MB


In [65]:
re.to_csv('yelp_review_clean.csv', index=False)

In [95]:
st = re.groupby(["business_id", "stars"]).count()

KeyError: ('IDtLPgUrqorrpqSLdfMhZQ', 1)

In [103]:
ints = [1,2,3,4,5]
print(len(st))
for i in bu["business_id"]:
    for j in ints:
        if (i,j) not in st.index:
            add = pd.Series([0], index=[(i,j)])
            st = pd.concat([st, add])
print(len(st))

2912
3355


In [104]:
fin = pd.merge(bu, st, how='inner', on='business_id')

In [120]:
print(st.index)

MultiIndex([('-3AooxIkg38UyUdlz5oXdw', 1),
            ('-3AooxIkg38UyUdlz5oXdw', 2),
            ('-3AooxIkg38UyUdlz5oXdw', 3),
            ('-3AooxIkg38UyUdlz5oXdw', 4),
            ('-3AooxIkg38UyUdlz5oXdw', 5),
            ('-6jvfSJGprbfBD2QrS9zQw', 5),
            ('-ALqLSTzkGDMscHdxA1NgA', 2),
            ('-ALqLSTzkGDMscHdxA1NgA', 3),
            ('-ALqLSTzkGDMscHdxA1NgA', 4),
            ('-ALqLSTzkGDMscHdxA1NgA', 5),
            ...
            ('wAPXYljzn9YDTvQqqL54pw', 2),
            ('HKHz6u48Qgftwk_VRkLqSA', 1),
            ('HKHz6u48Qgftwk_VRkLqSA', 2),
            ('HKHz6u48Qgftwk_VRkLqSA', 3),
            ('IRBhPAC4ZoDpXazpoB3epQ', 1),
            ('IRBhPAC4ZoDpXazpoB3epQ', 2),
            ('IRBhPAC4ZoDpXazpoB3epQ', 3),
            ('Bo-GGwQNcLVwyKeYkbfAYg', 1),
            ('Bo-GGwQNcLVwyKeYkbfAYg', 2),
            ('Bo-GGwQNcLVwyKeYkbfAYg', 3)],
           names=['business_id', 'stars'], length=3355)


In [122]:
print(st['-3AooxIkg38UyUdlz5oXdw', 1])

KeyError: ('-3AooxIkg38UyUdlz5oXdw', 1)

In [114]:
stars = {}
for key in st.index:
    print(key)
    if key[0] not in stars:
        stars[key[0]] = [0,0,0,0,0]

    stars[key[0]][key[1]] = st[key]
        
print(stars)

('-3AooxIkg38UyUdlz5oXdw', 1)


KeyError: ('-3AooxIkg38UyUdlz5oXdw', 1)